In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob as gb
import pandas as pd
import seaborn as sns

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers import Input
from tensorflow import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import models
from tensorflow.keras import optimizers
import json
import cv2
import seaborn as sns
from scipy.stats import norm
import statistics
import random
import shutil
import _utils as fn
%matplotlib inline

In [2]:
baseline = models.load_model('baseline_model2.h5')
vgg = models.load_model('VGG16_model.h5')
resnet = models.load_model('resnet_model.h5')

In [3]:
train_dir = './train4/'
valid_dir = './valid4/'

In [4]:
batch_size = 32
img_height = 100
img_width = 100

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle = True,
    color_mode="rgb",
    seed=42,
    subset='training')

validation_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle = True,
    color_mode="rgb",
    seed=42,
    subset='validation')

test_ds = test_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle = False,
    color_mode="rgb",
    seed=42,
    class_mode='sparse')

Found 14694 images belonging to 6 classes.
Found 3672 images belonging to 6 classes.
Found 4870 images belonging to 6 classes.


In [5]:
dict_class = train_ds.class_indices

In [6]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [7]:
from sklearn.metrics import top_k_accuracy_score
def TopKAccuracy(val_ds, test_ds, model):
    preds=model.predict(test_ds)  
    preds_val=model.predict(val_ds) 
    test_true = test_ds.labels
    val_true = val_ds.labels
    print('Top-1 (val):',top_k_accuracy_score(val_true, preds_val, k=1))
    print('Top-1 (test):',top_k_accuracy_score(test_true, preds, k=1))
    print('Top-5 (val):',top_k_accuracy_score(val_true, preds_val, k=5))
    print('Top-5 (test):',top_k_accuracy_score(test_true, preds, k=5))

In [8]:
TopKAccuracy(validation_ds, test_ds, baseline)

115/115 [==============================] - 18s 154ms/step
Top-1 (val): 0.17892156862745098
Top-1 (test): 0.7876796714579055
Top-5 (val): 0.8374183006535948
Top-5 (test): 0.999794661190965


In [10]:
TopKAccuracy(validation_ds, test_ds, vgg)

115/115 [==============================] - 20s 172ms/step
Top-1 (val): 0.16775599128540306
Top-1 (test): 0.946817248459959
Top-5 (val): 0.8273420479302832
Top-5 (test): 0.9995893223819302


In [11]:
TopKAccuracy(validation_ds, test_ds, resnet)

115/115 [==============================] - 22s 188ms/step
Top-1 (val): 0.16530501089324617
Top-1 (test): 0.971252566735113
Top-5 (val): 0.8259803921568627
Top-5 (test): 1.0


In [12]:
def predict_disease(image_path, model):
    img = cv2.imread(image_path)
    resized = cv2.resize(img,(100,100))
    img_array = tf.keras.utils.img_to_array(resized)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    label_names = ['โรคใบจุดมะเขือเทศ (Bacterial Spot Disease)',
                    'โรคใบไหม้ (Early Blight Disease)',
                    'ไม่พบโรค (Healthy)',
                    'โรคใบจุดวงกลม (Septoria Leaf Spot Disease)',
                    'โรคใบจุด (Target Spot Disease)',
                    'โรคใบหงิกเหลือง (Tomato Yellow Leaf Curl Virus Disease)']
    label=label_names[np.argmax(score)]
    return label

In [21]:
predict_disease('./testset_web/heal.jpg', resnet)

1/1 [==============================] - 0s 277ms/step


'โรคใบจุด (Target Spot Disease)'